In [ ]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ISR Dataset/IMDB Dataset.csv')

data['review'] = data['review'].str.lower()

In [ ]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because",
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should",
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [ ]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [ ]:
reviews_list = []
for i in range(len(data['review'])):
  reviews_list.append(data['review'][i])
  sentiment = data_without_stopwords['sentiment']

In [ ]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)


In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/ISR Dataset/glove.6B.50d.txt')

maxLen = 150

In [ ]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)


In [ ]:
from keras.layers import Bidirectional

In [ ]:
#model=imdb_rating(input_shape)
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X =Bidirectional(LSTM(128, return_sequences=True))(embeddings)

  X = Dropout(0.3)(X)

  X = Bidirectional(LSTM(128))(X)

  X = Dropout(0.3)(X)

  #X = LSTM(128)(X)
  #X = Dense(64)
  #X = Dropout(0.3)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
#model=imdb_rating(input_shape)
'''def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model'''

"def imdb_rating(input_shape):\n\n  X_indices = Input(input_shape)\n\n  embeddings = embedding_layer(X_indices)\n\n  X = LSTM(128, return_sequences=True)(embeddings)\n\n  X = Dropout(0.6)(X)\n\n  X = LSTM(128, return_sequences=True)(X)\n\n  X = Dropout(0.6)(X)\n\n  X = LSTM(128)(X)\n\n  X = Dense(1, activation='sigmoid')(X)\n\n  model = Model(inputs=X_indices, outputs=X)\n\n  return model"

In [ ]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           5612100   
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 256)          183296    
_________________________________________________________________
dropout (Dropout)            (None, 150, 256)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257   

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 129s 161ms/step - loss: 0.6514 - accuracy: 0.6119
Epoch 2/15
625/625 [==============================] - 94s 150ms/step - loss: 0.6189 - accuracy: 0.6573
Epoch 3/15
625/625 [==============================] - 90s 145ms/step - loss: 0.4465 - accuracy: 0.7986
Epoch 4/15
625/625 [==============================] - 88s 141ms/step - loss: 0.4024 - accuracy: 0.8170
Epoch 5/15
625/625 [==============================] - 86s 138ms/step - loss: 0.3842 - accuracy: 0.8300
Epoch 6/15
625/625 [==============================] - 85s 136ms/step - loss: 0.3586 - accuracy: 0.8446
Epoch 7/15
625/625 [==============================] - 84s 134ms/step - loss: 0.3331 - accuracy: 0.8567
Epoch 8/15
625/625 [==============================] - 84s 134ms/step - loss: 0.3082 - accuracy: 0.8674
Epoch 9/15
625/625 [==============================] - 83s 133ms/step - loss: 0.2983 - accuracy: 0.8702
Epoch 10/15
625/625 [==============================] - 83s 133ms/step - 

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 22s 64ms/step - loss: 0.3622 - accuracy: 0.8655


[0.3622434437274933, 0.8654999732971191]

In [ ]:
preds = model.predict(X_test_indices)

In [ ]:
n = np.random.randint(0,9999)

X_test[n]


'a serial killer dies in a snowstorm and gets mutated into frosty the snowman\'s evil twin. then goes on a killing spree. interesting plot. sounds scary. and it is scary. if you\'re five years old. otherwise, it\'s kind of cheesy. i saw it on cable and i\'m glad i didn\'t pay money to see it. it has all the charm and style of a low-budget movie which may become a cult film. i\'m sure it has a loyal fan base somewhere. i\'m just not in it. even though i didn\'t like the movie as a whole, there were some scenes i found amusing. such as the bathtub scene and the post-explosion scene with the picasso reference. it was also enjoyable to watch the many ways the heroes try to kill jack and he just doesn\'t seem to want to die. in short, "jack frost" is a good low-budget b-movie comedy, but a bad low-budget b-movie horror.'

In [ ]:
preds[n]

array([0.00520705], dtype=float32)

In [ ]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else:
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')


precicted sentiment : negative
correct sentiment : negative


In [ ]:
model.save_weights('/content/drive/MyDrive/ISR Dataset/imdb_weights_Glove_Bilstm.hdf5')

In [ ]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

In [ ]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data



In [ ]:
data = add_score_predictions(data, reviews_list_idx)

In [ ]:
data

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.694208,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.999623,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.996641,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.004695,negative
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.996964,positive
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.999873,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.000788,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.055050,negative
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.160507,negative
